In [ ]:
import ga_polynomial_simple as ga
import plotly.express as px
import numpy as np
import pandas as pd
import logging

In [ ]:
logging.getLogger().setLevel(logging.INFO)

In [ ]:
xRange = 20

yTarget = np.array([])

xValues = np.linspace(-1,1, xRange)

for x in xValues:
    yTarget = np.append(yTarget, 
                        25*x**5 + 18*x**4 + 31*x**3 - 14*x**2 + 7*x -19)

TARGET_COEFFS = [25,18,31,-14,7,-19]
TARGET_POINTS = yTarget
print("Global variables have been generated successfully for target points and target coefficient values.")

## Visualising Target Polynomial

In [ ]:
figTargetPolynomial = px.line(x=xValues, y=TARGET_POINTS, title='Target Polynomial')
figTargetPolynomial.show()

## Producing GA Predictions

#### Setup Parameters

In [ ]:
count = 500
individualLength = xRange
individualMin = min(TARGET_POINTS)*1.1
individualMax = max(TARGET_POINTS)*1.1

retain = 0.15
mutate = 0.01
random_select = 0.05

#### Running with selecion by roulette wheel

Specify GA training configuration

In [ ]:
maxGen = 401

Create first population and grade

In [ ]:
population = ga.createPopulation(count, individualMin, individualMax, individualLength)
fitness_history = [ga.getAvgPopulationError(population, target=TARGET_POINTS)]

Show some data for sanity check

In [ ]:
print("Average absolute error of initial population: {}".format(fitness_history[0]))

Create dataframe to store results

In [ ]:
comparison = pd.DataFrame()
comparison['target polynomial'] = TARGET_POINTS
comparison.name = 'Predicted vs. Target Polynomial'
comparison.index = xValues
comparison.index.name = 'x'
comparison.head()

Train GA for set configuration

In [ ]:
lastGrade = 1
gen=1

while lastGrade != 0 and len(fitness_history) < maxGen:

    population = ga.evolve(population, 
                           TARGET_POINTS, 
                           retain=retain, 
                           mutate=mutate, 
                           select='random')

    lastGrade = ga.getAvgPopulationError(population, TARGET_POINTS)
    fitness_history.append(lastGrade)
    
    if gen%200 == 0:
        comparison['gen_' + str(gen)] = population[0]
        comparison.head()

    gen += 1

In [ ]:
figTargetPolynomial = px.line(comparison, title='Predicted vs. Target Polynomial')
figTargetPolynomial.show()

In [ ]:
poppy = ga.createPopulation(20, 0, 21, 20)
poppy

In [ ]:
df = pd.DataFrame({'individual':poppy})
df

In [ ]:
df['grade'] = df.apply(lambda x: ga.getSumAbsError(x['individual'], TARGET_POINTS), axis=1)
df

In [ ]:
df = df.sort_values('grade')
df

In [ ]:
df['random'] = np.random.uniform(0.0, 1.0, len(df))
df

In [ ]:
mask = df['random'] > 0.5
mask

In [ ]:
df.loc[mask]

In [ ]:
a = [[3,4,5]]
a.append([99,88,77])
a

In [ ]:
retainLength = int(len(poppy)*retain)
retainLength

In [ ]:
df = pd.DataFrame({'individual': poppy})

# Create second column with grade for each individual in population
df['grade'] = df.apply(lambda x: ga.getSumAbsError(x['individual'], TARGET_POINTS), axis=1)

# Sort rows based on grade of individual
graded = df.sort_values('grade')

# Create list of best performers to retain
matingPool = graded.iloc[:retainLength, 0].tolist()

In [ ]:
graded

In [ ]:
matingPool

In [ ]:
# Select subset of rows from graded dataframe with remaining individuals
remaining = graded.iloc[retainLength:, :]

# Create new column with random float between range [0,1] for each remaining individual
remaining.insert(2, 'random', np.random.uniform(0.0, 1.0, len(remaining)))

In [ ]:
remaining.head()

In [ ]:
# Randomly select a subset of these individuals by comparing to random_select float
mask = remaining['random'] < 0.3
selected = remaining.loc[mask]

In [ ]:
selected

In [ ]:
# Cast column with individual's chromosome values to list to append to mating pool
matingPool.extend(selected['individual'].tolist())
matingPool

In [ ]:
selected['individual'].tolist()

In [ ]:
selected['individual']